In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import pandas as pd

In [ ]:
# Load the data
data = pd.read_csv('Anime.csv')
print(data.head())

#store the number of rows in the data
num_rows = data.shape[0]


In [ ]:
##Data Cleaning

#give each studio a unique id number and add it to the data
studios = data['Studio'].unique()
studio_ids = range(1, len(studios) + 1)
studio_dict = dict(zip(studios, studio_ids))
data['Studio_ID'] = data['Studio'].map(studio_dict)
print(data.head())



#drop the End_Year column
data = data.drop('End_year', axis=1)
data = data.drop('staff', axis=1)
data = data.drop('Voice_actors', axis=1)
data = data.drop('Description', axis=1)
data = data.drop('Name', axis=1)
data = data.drop('Japanese_name', axis=1)
data = data.drop('Type', axis=1)
data = data.drop('Tags', axis=1)
data = data.drop('Related_Mange', axis=1)
data = data.drop('Related_anime', axis=1)
data = data.drop('Rank', axis=1)
data = data.drop('Release_season', axis=1)
data = data.drop('Release_year', axis=1)
data = data.drop('Studio', axis=1)
data = data.drop('Content_Warning', axis=1)

#print the names of the columns in the data
print(data.columns)

#Remove the rows with missing values
data = data.dropna()

#store and print the number of rows after removing the missing values
num_rows_cleaned = data.shape[0]
print(num_rows_cleaned)

print("=====================")

##Indexing

#Give every show a unique id number
data['ID'] = range(1, num_rows_cleaned + 1)
print(data.head())



In [ ]:
# Convert string values to numeric values
data = data.apply(pd.to_numeric, errors='coerce')

# Plot the correlation matrix
correlation_matrix = data.corr()
plt.matshow(correlation_matrix)
plt.xticks(range(len(correlation_matrix.columns)), correlation_matrix.columns, rotation='vertical')
plt.yticks(range(len(correlation_matrix.columns)), correlation_matrix.columns)
plt.colorbar()
plt.show()

